In [1]:
import random
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [3]:
# class Particle():
#     def __init__(self):
#         self.position = np.array([(-1) ** (bool(random.getrandbits(1))) 
#                                   * random.random() * 50, 
#                                   (-1)**(bool(random.getrandbits(1)))
#                                  * random.random()*50])
#         self.pbest_position = self.position
#         self.pbest_value = float('inf')
#         self.velocity = np.array([0, 0])
    
#     def __str__(self):
#         print("I am at ", self.position, " meu pbest is ", self.pbest_position)
    
#     def move(self):
#         self.position = self.position + self.velocity
    
# class Space():
    
#     def __init__(self, target, target_error, n_particles):
#         self.target = target
#         self.target_error = target_error
#         self.n_particles = n_particles
#         self.particles = []
        
#         self.gbest_value = float('inf')
#         self.gbest_position = np.array([random.random() * 50, random.random() * 50])
    
#     def print_particles(self):
#         for particle in self.particles:
#             particle.__str__()
    
#     def fitness(self, particle):
#         return particle.position[0] ** 2 + particle.position[1] ** 2 + 1
    
#     def set_pbest(self):
#         for particle in self.particles:
#             fitness_cadidate = self.fitness(particle)
            
#             if(particle.pbest_value > fitness_cadidate):
#                 particle.pbest_value = fitness_cadidate
#                 particle.pbest_position = particle.position
    
#     def set_gbest(self):
#         for particle in self.particles:
#             best_fitness_cadidate = self.fitness(particle)
#             if(self.gbest_value > best_finess_cadidate):
#                 self.gbest_value = best_finess_cadidate
#                 self.gbest_position = particle.position
                
#     def move_particles(self):
#         for particle in self.particles:
#             global W
#             new_velocity = (W*particle.velocity) + (c1*random.random()) * (particle.pbest_position - particle.position) + \
#                             (random.random()*c2) * (self.gbest_position - particle.position)
#             particle.velocity = new_velocity
#             particle.move()
        

In [4]:
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))

In [5]:
# 
def sigmoid_function(particle):
    return 1 / (1+math.exp(-particle))

particle = [1, 0, 0, 0, 1, 0, 1, 0]
sigmoid = sigmoid_function(-1)
sigmoid

0.2689414213699951

In [9]:
def fitness_function(particle_position, X, y, x_test, y_test):
    particle_position = particle_position.tolist()
    
    if(particle_position.count(0) != len(particle_position)):
        # get index with value 0
        cols = [index for index in range(
            len(particle_position)) if particle_position[index] == 0]

        # get features subset
        X_parsed = X.drop(X.columns[cols], axis=1)
        x_test_parsed = x_test.drop(X.columns[cols], axis=1)
        X_subset = pd.get_dummies(X_parsed)
        x_test_subset = pd.get_dummies(x_test_parsed)
        
        # apply classification algorithm
        clf = LogisticRegression().fit(X_subset, y)
        y_pred = clf.predict(x_test_subset)
        #score = clf.score(x_test, y_test)
        mse = mean_squared_error(y_test, y_pred)
        # print(particle_position , " -> ", mse)
        # return (avg(cross_val_score(clf, X_subset, y, cv=5)),)
        # return particle_position[0]**2 + particle_position[1]**2 + 1
        return mse
    else:
        return(1)
    
    #return particle_position[0]**2 + particle_position[1]**2 + 1

In [10]:
df = pd.read_csv('datasets/iris.csv',)
le = LabelEncoder()
le.fit(df.iloc[:, -1])
y = le.transform(df.iloc[:, -1]) # label
X = df.drop([df.columns[0], 'Species'], axis=1)

from sklearn.model_selection import train_test_split

x_train,x_test,y_train, y_test = train_test_split(X , y , test_size=0.4 , random_state=0)

# get accuracy with all features
all_particle = [1 for i in range(len(x_train.columns))] # true column (feature)
particle_position = [np.random.randint(2, size=len(x_train.columns)) for _ in range(n_particles)]
pbest_position = particle_position
pbest_fitness_value = np.array([float('inf') for _ in range(n_particles)])
gbest_fitness_value = float('inf')
gbest_position = np.array([float('inf'), float('inf'), float('inf'), float('inf')])
velocity_vector = ([np.array([0, 0, 0, 0]) for _ in range(n_particles)])

c1 = 0.8
c2 = 0.9
n_iterations = 50
target_error = 0.00006
n_particles = 30
W = 0.5
iteration = 0 
n_iterations = 5

while(iteration < n_iterations):
    for i in range(n_particles):
        
        fitness_cadidate = fitness_function(particle_position[i], x_train, y_train, x_test, y_test)
        # print(fitness_cadidate)
        if(pbest_fitness_value[i] > fitness_cadidate):
            pbest_fitness_value[i] = fitness_cadidate
            pbest_position[i] = particle_position[i]
        
        if(gbest_fitness_value > fitness_cadidate):
            print("update from :", gbest_fitness_value, " to ", fitness_cadidate)
            gbest_fitness_value = fitness_cadidate
            gbest_position = particle_position[i]
            
    for i in range(n_particles):
        # velocity
        new_velocity = (W*velocity_vector[i]) + (c1*random.random()) * (pbest_position[i] - particle_position[i]) + (c2*random.random()) * (gbest_position-particle_position[i])
        # position
        new_position = new_velocity + particle_position[i]
        # sigmoid
        new_position = [0 if np.random.uniform(0, 1)>= sigmoid_function(x) else 1 for x in new_position]
        particle_position[i] = np.asarray(new_position)
        
    
    print("-------> ", iteration, "<--------")
    iteration += 1

print("The best solution is : ", gbest_position, " in n_iteration: ", iteration)


update from : inf  to  0.183333333333
update from : 0.183333333333  to  0.133333333333
update from : 0.133333333333  to  0.1
update from : 0.1  to  0.0833333333333
------->  0 <--------
update from : 0.0833333333333  to  0.0666666666667
------->  1 <--------
------->  2 <--------
------->  3 <--------
------->  4 <--------
The best solution is :  [0 1 1 1]  in n_iteration:  5


In [8]:
# # search_space = Space(1, target_error, n_particles)
# # particle_vector = [Particle() for _ in range(search_space.n_particles)]
# # search_space.particle = particle_vector
# def fitness_function(position):
#     return position[0]**2 + position[1]**2 + 1

# particle_position_vector = np.array([np.array([(-1) ** (bool(random.getrandbits(1))) * random.random()*50, (-1)**(bool(random.getrandbits(1))) * random.random()*50]) for _ in range(n_particles)])
# print(type(particle_position_vector))
# pbest_position = particle_position_vector
# pbest_fitness_value = np.array([float('inf') for _ in range(n_particles)])

# gbest_fitness_value = float('inf')
# gbest_position = np.array([float('inf'), float('inf')])

# velocity_vector = ([np.array([0, 0]) for _ in range(n_particles)])
# target = 1
# W = 0.5


# iteration = 0
# while(iteration < n_iterations):
#     for i in range(n_particles):
#         # search_space.set_pbest()
#         fitness_cadidate = fitness_function(particle_position_vector[i])
    
#         # search_space.set_gbest()
        
#         if(pbest_fitness_value[i] > fitness_cadidate):
#             pbest_fitness_value[i] = fitness_cadidate
#             pbest_position[i] = particle_position_vector[i]
        
#         if(gbest_fitness_value > fitness_cadidate):
#             gbest_fitness_value = fitness_cadidate
#             gbest_position = particle_position_vector[i]
    

#     if(abs(gbest_fitness_value - target) < target_error):
#         break
        
#     # search_space.move_particles()
#     for i in range(n_particles):
#         new_velocity = (W*velocity_vector[i]) + (c1*random.random()) * (pbest_position[i] - particle_position_vector[i]) + (c2*random.random()) * (gbest_position-particle_position_vector[i])
                
#         new_position = new_velocity + particle_position_vector[i]
#         particle_position_vector[i] = new_position
    
#     print("-------> ", iteration, "<--------")
#     iteration += 1

# print("The best solution is : ", gbest_position, " in n_iteration: ", iteration)